In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# 1 Least squares and linear basis functions models
## 1.1 Least squares

In [2]:
def least_squares(y, tx):
    x = tx#tx[:,1:tx.shape[1]]
    gram = x.T @ x
    if np.linalg.det(gram)!=0:
        w = np.linalg.inv(gram)@x.T@y
    else:
        w = np.linealt.solv(gram,x.T@y)
    e = y - x@w
    MSE = 1/(2*y.shape[0])*(e.T@e)
    return MSE,w

### Load the data
Here we will reuse the dataset `height_weight_genders.csv` from previous exercise section to check the correctness of your implementation. Please compare it with your previous result.

In [3]:
print("AHAHAH")

AHAHAH


In [4]:
from helpers import *
from grid_search import *

def test_your_least_squares():
    height, weight, gender = load_data_from_ex02(sub_sample=False, add_outlier=False)
    x, mean_x, std_x = standardize(height)
    y, tx = build_model_data(x, weight)
    grid_w0, grid_w1 = generate_w(num_intervals=50)
    grid_losses = grid_search(y, tx, grid_w0, grid_w1)
    _, w0,w1 = get_best_parameters(grid_w0, grid_w1, grid_losses)
    _,wLS = least_squares(y,tx)
    print(wLS)
    return w0, w1, wLS

Test it here

In [5]:
test_your_least_squares()

NotImplementedError: 

## 1.2 Least squares with a linear basis function model
Start from this section, we will use the dataset `dataEx3.csv`.

### Implement polynomial basis functions

In [ ]:
# load dataset
x, y = load_data()
print("shape of x {}".format(x.shape))
print("shape of y {}".format(y.shape))

In [ ]:
def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree."""
    result = np.zeros((degree+1,len(x)))
    for i in range(degree+1):
        temp = np.power(x,i)
        result[i] = temp
    return result.T

Let us play with polynomial regression. Note that we will use your implemented function `compute_mse`. Please copy and paste your implementation from exercise02.

In [ ]:
from plots import *

def polynomial_regression():
    """Constructing the polynomial basis function expansion of the data,
       and then running least squares regression."""
    # define parameters
    degrees = [1, 3, 7, 12]
    
    # define the structure of the figure
    num_row = 2
    num_col = 2
    f, axs = plt.subplots(num_row, num_col)

    for ind, degree in enumerate(degrees):
        print(ind)
        phi = build_poly(x, degrees[ind])
        #print(x.shape)
        #print(phi.shape)
        # ***************************************************
        # INSERT YOUR CODE HERE
        # form the data to do polynomial regression.: TODO
        # ***************************************************
        MSE, weights = least_squares(y,phi)
        rmse = np.sqrt(2*MSE)
        # ***************************************************
        # INSERT YOUR CODE HERE
        # least square and calculate RMSE: TODO
        # ***************************************************

        print("Processing {i}th experiment, degree={d}, rmse={loss}".format(
              i=ind + 1, d=degree, loss=rmse))
        # plot fit
        plot_fitted_curve(
            y, x, weights, degree, axs[ind // num_col][ind % num_col])
    plt.tight_layout()
    plt.savefig("visualize_polynomial_regression")
    plt.show()

Run polynomial regression

In [6]:
polynomial_regression()

NameError: name 'polynomial_regression' is not defined

# 2 Evaluating model predication performance


Let us show the train and test splits for various polynomial degrees. First of all, please fill in the function `split_data()`

In [ ]:
def split_data(x, y, ratio, seed=1):
    """
    split the dataset based on the split ratio. If ratio is 0.8 
    you will have 80% of your data set dedicated to training 
    and the rest dedicated to testing
    """
    # set seed
    np.random.seed(seed)
    packed =  np.vstack([y,x]).T
    np.random.shuffle(packed)
    N = y.shape[0]
    eightyN = int(ratio*N)
    xTrain = packed[0:eightyN,1]
    yTrain = packed[0:eightyN,0]
    xTest = packed[eightyN:N, 1]
    yTest = packed[eightyN:N,0]
    # ***************************************************
    # INSERT YOUR CODE HERE
    # split the data based on the given ratio: TODO
    # ***************************************************
    return xTrain, yTrain, xTest, yTest

Then, test your `split_data` function below.

In [ ]:
def train_test_split_demo(x, y, degree, ratio, seed):
    """polynomial regression with different split ratios and different degrees."""
    xTrain, yTrain, xTest, yTest = split_data(x,y,ratio,seed)
    # ***************************************************
    # INSERT YOUR CODE HERE
    # split the data, and return train and test data: TODO
    # ***************************************************
    phiTrain = build_poly(xTrain, degree)
    phiTest = build_poly(xTest, degree)
    # ***************************************************
    # INSERT YOUR CODE HERE
    # form train and test data with polynomial basis function: TODO
    # ***************************************************
    MSE,weights = least_squares(yTrain, phiTrain)
    # ***************************************************
    # INSERT YOUR CODE HERE
    # calculate weight through least square: TODO
    # ***************************************************
    rmse_tr = np.sqrt(2*MSE)
    rmse_te = np.sqrt(2*compute_loss(yTest, phiTest, weights))
    # ***************************************************
    # INSERT YOUR CODE HERE
    # calculate RMSE for train and test data,
    # and store them in rmse_tr and rmse_te respectively: TODO
    # ***************************************************
    print("proportion={p}, degree={d}, Training RMSE={tr:.3f}, Testing RMSE={te:.3f}".format(
          p=ratio, d=degree, tr=rmse_tr, te=rmse_te))


In [ ]:
seed = 6
degrees = [1, 3, 7, 12]
split_ratios = [0.9, 0.5, 0.1]

for split_ratio in split_ratios:
    for degree in degrees:
        train_test_split_demo(x, y, degree, split_ratio, seed)

# 3 Ridge Regression
Please fill in the function below.

In [ ]:
def ridge_regression(y, tx, lambda_):
    #Using L2
    lambdaPrime = lambda_*(2*tx.shape[0])
    x = tx
    gramLambda = x.T@x + np.eye(x.shape[1])*lambdaPrime
    if np.linalg.det(gramLambda)!=0:
        w = np.linalg.inv(gramLambda)@x.T@y
    else:
         w = np.linealt.solv(gramLambda,x.T@y)
    return w


In [ ]:
def ridge_regression_demo(x, y, degree, ratio, seed):
    """ridge regression demo."""
    # define parameter
    lambdas = np.logspace(-5, 0, 15)
    xTrain, yTrain, xTest, yTest = split_data(x,y,ratio,seed)

    # ***************************************************
    # INSERT YOUR CODE HERE
    # split the data, and return train and test data: TODO
    # ***************************************************
    phiTrain = build_poly(xTrain, degree)
    phiTest = build_poly(xTest, degree)
    # ***************************************************
    # INSERT YOUR CODE HERE
    # form train and test data with polynomial basis function: TODO
    # ***************************************************
    rmse_tr = []
    rmse_te = []
    for ind, lambda_ in enumerate(lambdas):
        weights = ridge_regression(yTrain, phiTrain,lambda_)
        
        # ***************************************************
        # INSERT YOUR CODE HERE
        # ridge regression with a given lambda
        # ***************************************************
        print(ind)
        rmse_tr.append(np.sqrt(2*compute_loss(yTrain,phiTrain,weights)))
        rmse_te.append(np.sqrt(2*compute_loss(yTest,phiTest,weights)))
        print("proportion={p}, degree={d}, lambda={l:.3f}, Training RMSE={tr:.3f}, Testing RMSE={te:.3f}".format(
               p=ratio, d=degree, l=lambda_, tr=rmse_tr[ind], te=rmse_te[ind]))
        
    # Plot the obtained results
    plot_train_test(rmse_tr, rmse_te, lambdas, degree)



In [ ]:
seed = 56
degree = 7
split_ratio = 0.5
ridge_regression_demo(x, y, degree, split_ratio, seed)